<a href="https://colab.research.google.com/github/sarwar-ku/data_science/blob/main/Image_classification_YOLO5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    from google.colab import drive
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

Note: using Google CoLab
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


Part 6.5: Recognizing Multiple Images with YOLO5
Programmers typically design convolutional neural networks to classify a single item centered in an image. However, as humans, we can recognize many items in our field of view in real-time. It is advantageous to recognize multiple items in a single image. One of the most advanced means of doing this is YOLOv5. You Only Look Once (YOLO) was introduced by Joseph Redmon, who supported YOLO up through V3. [Cite:redmon2016you] The fact that YOLO must only look once speaks to the efficiency of the algorithm. In this context, to "look" means to perform one scan over the image. It is also possible to run YOLO on live video streams.

Joseph Redmon left computer vision to pursue other interests. The current version, YOLOv5 is supported by the startup company Ultralytics, who released the open-source library that we use in this class.[Cite:zhu2021tph]

Researchers have trained YOLO on a variety of different computer image datasets. The version of YOLO weights used in this course is from the dataset Common Objects in Context (COCO). [Cite: lin2014microsoft] This dataset contains images labeled into 80 different classes. COCO is the source of the file coco.txt used in this module.

Using YOLO in Python
To use YOLO in Python, we will use the open-source library provided by Ultralytics.

YOLOv5 GitHub
The code provided in this notebook works equally well when run either locally or from Google CoLab. It is easier to run YOLOv5 from CoLab, which is recommended for this course.

We begin by obtaining an image to classify.

In [5]:
import urllib.request
import shutil
from IPython.display import Image
!mkdir /content/images/

URL = "https://github.com/sarwar-ku/data_science/blob/main/otherfiles/jeff_cook.jpg"
LOCAL_IMG_FILE = "/otherfiles/otherfiles/jeff_cook.jpg"

with urllib.request.urlopen(URL) as response, \
  open(LOCAL_IMG_FILE, 'wb') as out_file:
    shutil.copyfileobj(response, out_file)

Image(filename=LOCAL_IMG_FILE)

mkdir: cannot create directory ‘/content/images/’: File exists


FileNotFoundError: [Errno 2] No such file or directory: '/otherfiles/otherfiles/jeff_cook.jpg'

Installing YOLOv5
YOLO is not available directly through either PIP or CONDA. Additionally, YOLO is not installed in Google CoLab by default. Therefore, whether you wish to use YOLO through CoLab or run it locally, you need to go through several steps to install it. This section describes the process of installing YOLO. The same steps apply to either CoLab or a local install. For CoLab, you must repeat these steps each time the system restarts your virtual environment. You must perform these steps only once for your virtual Python environment for a local install. If you are installing locally, install to the same virtual environment you created for this course. The following commands install YOLO directly from its GitHub repository.

In [ ]:
import sys

!git clone https://github.com/ultralytics/yolov5 --tag 6.2  # clone
!mv /content/6.2 /content/yolov5
%pip install -qr /content/yolov5/requirements.txt  # install
sys.path.insert(0,'/content/yolov5')

import torch
import utils
display = utils.notebook_init()  # checks

Next, we will run YOLO from the command line and classify the previously downloaded kitchen picture. You can run this classification on any image you choose.

In [ ]:
# Prepare directories for YOLO command line
!rm -R /content/yolov5/runs/detect/*
!mkdir /content/images
!cp /content/street/jeff_cook.jpg /content/images

# Run YOLO to classify
!python /content/yolov5/detect.py --weights yolov5s.pt --img 1024 \
  --conf 0.25 --source /content/images/

# Display the images
from IPython.display import Image

URL = '/content/yolov5/runs/detect/exp/jeff_cook.jpg'
Image(filename=URL, width=300)

Running YOLOv5
In addition to the command line execution, we just saw. The following code adds the downloaded YOLOv5 to Python's environment, allowing yolov5 to be imported like a regular Python library.

In [ ]:
import torch

# Model
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or yolov5n - yolov5x6, custom

# Inference
results = yolo_model(LOCAL_IMG_FILE)

# Results
df = results.pandas().xyxy[0]
df

It is important to note that the yolo class instantiated here is a callable object, which can fill the role of both an object and a function. Acting as a function, yolo returns a Pandas dataframe that contains the bounding boxes (xmin/xmax and ymin/ymax), confidence, and name/class of each item detected.

Your program should use these values to perform whatever actions you wish due to the input image. The following code displays the images detected above the threshold.

You can obtain the counts of images through the use of a Pandas groupby and pivot.

In [ ]:
df2 = df[['name','class']].groupby(by=["name"]).count().reset_index()
df2.columns = ['name','count']
df2['image'] = 1
df2.pivot(index=['image'],columns='name',values='count').reset_index().fillna(0)